In [1]:
import logging
import logging.config
import os
import json
# import ConfigParser
import sys
import re
from glob import glob
import csv

import configuration
from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError

from humanfriendly import prompts

from progressbar import AnimatedMarker, Bar, BouncingBar, Counter, ETA, \
    AdaptiveETA, FileTransferSpeed, FormatLabel, Percentage, \
    ProgressBar, ReverseBar, RotatingMarker, \
    SimpleProgress, Timer, UnknownLength

# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [2]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [3]:
def fileSearch(path = None, search = None):
        '''search for a file name string in a given path'''
        if not search:
            search = ''
        logger = logging.getLogger(__name__)
        logger.debug('searching path: {0} for \"{1}\"'.format(path, search))
        try:
            searchPath = os.path.expanduser(path)
        except AttributeError as e:
            logger.error('Error: no path specified')
            searchPath = ''
        try:
            allFiles = os.listdir(searchPath)
        except OSError as e:
            logger.error(e)
            return(None)
        
        # list comprehension search with regex
        #http://www.cademuir.eu/blog/2011/10/20/python-searching-for-a-string-within-a-list-list-comprehension/
        regex = re.compile('.*('+search+').*')
        return([m.group(0) for l in allFiles for m in [regex.search(l)] if m])

In [4]:
def getConfiguration(cfgfile):
    # required configuraiton options
    # Section: {'option': 'default value'}
    cfgpath = os.path.dirname(cfgfile)
    config_required = {
        'Main': {'credentials': os.path.join(cfgpath, 'credentials/'), 
                 'teamdriveID': '',
                 'teamdrivename': '',
                 'folderID': '',
                 'foldername': '',
                 },
        }

    config = configuration.get_config(cfgfile)

    update_config = False

    for section, values in config_required.items():
        if not config.has_section(section):
            logger.warn('section: {} not found in {}'.format(section, cfgfile))
            logger.debug('adding section {}'.format(section))
            config.add_section(section)
            update_config = True
        for option, value in values.items():
            if not config.has_option(section, option):
                logger.warn('option: {} not found in {}'.format(option, cfgfile))
                logger.debug('adding option {}: {}'.format(option, value))

                config.set(section, option, value)
                update_config = True


    # for section, options in config_required.items():

    if update_config:
        try:
            logger.debug('updating configuration file at: {}'.format(cfgfile))
            configuration.create_config(cfgfile, config)
        except Exception as e:
            logger.error(e)
            
    return(config)

In [5]:
def getTeamDrive(myDrive):
    '''
    menu driven interaction for choosing a writeable team drive
    '''
    #available and writeable team drives
    logger = logging.getLogger(__name__)
    try:
        teamdrives_all = myDrive.listTeamDrives()
    except Exception as e:
        logger.error(e)
    
    if len(teamdrives_all) < 1:
        logger.warn('no writeable team drives found for this user')
        return(None)
    
    teamdrives_writeable = {}
    for drive in teamdrives_all:
        if drive['capabilities']['canAddChildren']:
            teamdrives_writeable[drive['name']] = drive
    
    teamdrive = None
    
    try:
        teamdrive_name = prompts.prompt_for_choice(choices=teamdrives_writeable, default=None)
        teamdrive=teamdrives_writeable[teamdrive_name]
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
    
    return(teamdrive)

In [6]:
def getPortfolioFolder(myDrive):
    '''
    menu driven interaction to locate and choose portfolio folder on team drive
    
    returns:
        folder (dictionary) - google drive object properties dictionary
    '''
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive: {}; {}'.format(teamdriveName, teamdriveID))
    
    foldersearch = ''
    attempt = 5
    while (len(foldersearch) <= 0) and (attempt > 0):
        attempt -= 1
        foldersearch = prompts.prompt_for_input(question='Please enter a portion of the portfolio folder name (case insensitive): ',
                                                default=None, strip=True)
        if len(foldersearch) <= 0:
            print 'Nothing entered. {} attempts remain.'.format(attempt)
            
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        if len(searchresult['files']) <= 0:
            print 'No folders contained "{}"'.format(foldersearch)
            print 'Please try entering only part of the folder name'
            foldersearch = ''
            
    
    
    if foldersearch <= 0:
        logger.warn('user entered 0 length string for folder search')
        return(None)
    
    
    foldernames = []
    for each in searchresult['files']:
        foldernames.append(each['name'])
    
    try:
        foldername = prompts.prompt_for_choice(choices=foldernames)
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
        return(None)
    
    return(searchresult['files'][foldernames.index(foldername)])
    

In [7]:
def getPathfromList(list_path=['~/'], message='Choose from the paths below', default=None):
    '''
    menu driven interaction to select path 
    accepts:
        list_path (list) - list of paths to display
        messager (string) - message to display
        default (string) - default choice (must be one of the items in the list otherwise defaults to None)
    
    returns:
        searchPath (string) - valid and existing path
    '''
    logger = logging.getLogger(__name__)
    logger.debug('path list: {}'.format(list_path))
    searchPath = None
    list_path = sorted(list_path)
    list_path.append('OTHER')
    
    if default not in list_path:
        logging.warn('default ({}) not in list; setting default = None'.format(default))
        default = None
    
    if message:
        print message
    searchPath = prompts.prompt_for_choice(choices=list_path, default=default)
    logger.debug('searchPath = {}'.format(searchPath))
    
    if not searchPath is 'OTHER':
        searchPath = os.path.expanduser(searchPath)
    else:
        attempt = 5
        searchPath = None
        while not searchPath and attempt > 0:
            logger.debug('attempts remaining: {}'.format(attempt))
            attempt -= 1
            searchPath = prompts.prompt_for_input('Please type the complete path to use: ')
            logger.debug('searchPath = {}'.format(searchPath))         
    
    if len(searchPath) < 1:
        searchPath = None
    return (searchPath) 

In [8]:
def getFiles(path='~/', pattern='.*', ignorecase=True):
    '''
    search path for files matching pattern (regular expression)
    accepts:
        path (string) - path to search
        pattern (string) - regular expression to search for in file names
        ignorecase (bool) - defaults to True, ignore file name case'''
    logger = logging.getLogger(__name__)
    
    path = os.path.expanduser(path+'/*')
    logger.debug('Path: {}, glob pattern: {}, ignorecase: {}'.format(path, pattern, ignorecase))
    
    flags = re.IGNORECASE
    files = []
        
    if ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f, flags=flags)]:
            files.append(eachfile)
    
    if not ignorecase:
        for eachfile in [f for f in glob(path) if re.match(pattern, f)]:
            files.append(eachfile)        
    
    return(files)

In [9]:
def chooseFile(path='~/', pattern='.*', ignorecase=True, message='Please choose a file from the list'):
    '''
    menu interaction for choose a file in the specified path from the file glob created using a regex pattern
    accepts:
        path (string) - path to search
        pattern (string) - regular expression to search for in file names
        ignorecase (bool) - defaults to True, ignore file name case
        message(string) - message to show
    returns:
        selection (string) - selected file'''
    
    logger = logging.getLogger(__name__)
    filelist = getFiles(path=path, pattern=pattern, ignorecase=ignorecase)
    logger.debug('filelist: {}'.format(filelist))
    print message
    selection = prompts.prompt_for_choice(choices=filelist)
    return(selection)

In [10]:
def fileToList(inputfile):
    logger = logging.getLogger(__name__)
    logger.debug('inputfile = {}'.format(inputfile))
    outputlist = []
    
    try:
        with open(inputfile, 'r') as fhandle:
            logger.debug('reading file: {}'.format(inputfile))
            for line in fhandle:
                outputlist.append(line.strip('\n'))
    except IOError as e:
        logger.debug(e)
    return(outputlist)

In [11]:
def checkFolder(folderID, myDrive):
    '''
    Checks properties of a folder id

    returns:
        tuple(isFolder (bool), writeable(bool), properties(dict)) - false if no match
    
    '''
    logger = logging.getLogger(__name__)
    logger.debug('checking folder id: {}'.format(folderID))
    props = None
    isFolder = False
    writeable = False
    try:
        logger.debug('checking properties for ID: {}'.format(folderID))
        props = myDrive.getprops(folderID, 'capabilities, mimeType, name')
    except GDriveError as e:
        logger.error('failed to get properties for {}; {}'.format(folderID, e))
        logger.error('please try again. if this error persists, please try reconfiguring the folder')
        logger.error('cannot continue; exiting')
    
    try:
        if props['mimeType'] == myDrive.mimeTypes['folder']:
            isFolder = True
            logger.debug('this ID is a folder')
        if props['capabilities']['canAddChildren']:
            writeable = True
            logger.debug('this ID is a writable folder')
        else:
            logger.debug('this ID is not a writable folder')
    except (KeyError, TypeError) as e:
        pass
    
    if not props:
        logger.warn('this ID is not a folder')
                
    return(isFolder, writeable, props)

In [12]:
def mapHeaders(file_csv, expected_headers=[]):
    '''map an expected list of header values to their position in a csv
    accepts:
        file_csv (csv.reader object) 
        expected_headers (list) - list of headers to search for, ignoring all others
    '''
    logger = logging.getLogger(__name__)
    logger.debug('mapping headers')
    
    missing_headers = []
    headerMap = {} 
    
    try:
        csvHeader = file_csv[0]
    except IndexError as e:
        logging.error('csv empty: {}'.format(e))
        return(None)
    
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
    
    headerMap['missingheaders'] = missing_headers
    
    headerMap['headers'] = {}

    if len(missing_headers) > 0:
        logging.warn('missing headers: {}'.format(missing_headers))
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            headerMap['headers'][value] = index
    logger.debug('successfully maped headers')
    return(headerMap)

In [36]:
def createFolders(myDrive, parentFolder, teamdriveID, studentexport_csv, header_map):
    '''
    create folders on google drive if they do not exist
    
    accepts:
        myDrive(googleDrive object)
        folderid (string) - folder id string
        folders (list) - list of folder names
    
    returns:
        folderinfo (dict) - {folder name: folder url, or None for failures}
    '''
    logger = logging.getLogger(__name__)
    logger.debug('creating folders at gdrive path: {}'.format(folderID))
    
    #TO DO: wtf is this?
    myDrive.getprops
    
    studentFolders = {}
    classOfFolders = {}
    classOf_string = 'Class Of-'
    
    
    # Define progress bar widgets for consistent look and feel
    widgets = ['Checking: ', Percentage(), ' ',
               Bar(marker='=',left='[',right=']'), 
               ' Processed ', Counter(),
               ' ', ETA()]

    
    # check that google drive parent folder exists and is a folder
    logger.debug('checking that portfolio folder exists and is writeable: {}'.format(parentFolder))
    try:
        folder_check = checkFolder(parentFolder, myDrive)
    except Exception as e:
        logger.error('fatal error: {}'.format(e))
        return(None)
    if all(folder_check):
        logger.debug('folder is OK')
    else:
        if not folder_check[1]:
            print 'The portfolio folder you selected is not writeable'
            logger.error('{} not writeable'.format(parentFolder))
            return(None)
        if folder_check[2]['mimeType'] != myDrive.mimeTypes['folder']:
            print 'It appears you have not selected a valid folder. Please consider reconfiguring.'
            logger.error('{} not a folder'.format(parentFolder))
            return(None)
    
    # gather all unique and classOf rows from csv and build dictionaries
    logger.debug('gathering "Class Of" folders from student export file')
  
    # build dictionaries 
    for student in studentexport_csv[1:]:
        classOf = student[headerMap['headers']['ClassOf']]
        studentNumber = student[headerMap['headers']['Student_Number']]
        
        classOfFolders[classOf] = {}
        classOfFolders[classOf]['name'] = classOf_string + classOf
    
    logger.debug('{} "Class Of" folders found in student export file'.format(len(classOfFolders)))
    
    # check for existence of Class Of folders and create if they are missing
    logger.debug('checking for existing "Class Of" folders in google drive')
    
    # init progress bar for class of folders 
    print 'Checking for or creating {} "Class Of-" folders in portfolio folder'.format(len(classOfFolders))
    pbar = ProgressBar(widgets=widgets, maxval=len(classOfFolders))
    pbar.start()
    pbar_index = 0
    
    for eachClass in classOfFolders.keys():
        # update the progress bar
        pbar.update(pbar_index)
        pbar_index += 1
        
        # assume folder needs to be created
        createfolder=True
        folder_name = classOfFolders[eachClass]['name']
                
        result = {}
        try:
            result = myDrive.search(name=folder_name, mimeType='folder', 
                                   teamdrive=teamdriveID, trashed=False,
                                   fields='name, createdTime, mimeType, parents, properties, id',
                                   orderBy='createdTime')     
        except GDriveError as e:
            logger.error('failed to find folder: {}'.format(e))
        except Exception as e:
            logger.error('fatal error: {}'.format(e))
            doExit(0)

            
        # check for a valid files return, use the first entry
        if result.has_key('files'):
            if len(result['files']) > 0:
                logger.debug('folder found; checking if valid and wirteable')
                
                try: 
                    if all(checkFolder(result['files'][0]['id'], myDrive)):
                        logger.debug('folder OK')
                        classOfFolders[eachClass] = result['files'][0]
                        createfolder=False
                except Exception as e:
                    logger.error('fatal error: {}'.format(e))
                    doExit(0)
                    
            if len(result['files']) > 1:
                logger.debug('{} {} folders found. Using the oldest.'.format(len(result['files']), folder_name))
                print 'there are multiple folders with the name {}; consider deleting the uneeded folders.'.format(folder_name)
                    
        if createfolder:
            logger.debug('creating folder: {}'.format(folder_name))
            try:
                result = myDrive.add(name=folder_name, parents=parentFolder, mimeType='folder')
            except GDriveError as e:
                logger.error('failed to create folder {} with error {}'.format(folder_name, e))
                classOfFolders[eachClass] = None
                continue
            except Exception as e:
                logger.error('fatal error: {}'.format(e))
                doExit(0)
            
            # check for a result from the creation and record it
            if any(result):
                classOfFolders[eachClass] = result
            else:
                classOfFolders[eachClass] = None
                
                
#         # record the information from the created folder
#         if len(result['files']) > 0:
#             classOfFolders[eachClass] = result['files'][0]
#         else:
#             classOfFolders[eachClass] = None

    # finish updating the pbar for Class Of- Folders
    pbar.finish()
    print 'Completed\n'

    # create student folders within the class of folders
    # init progress bar for class of folders 
    print 'Checking for or creating {} student folders in "Class Of-" folders'.format(len(studentexport_csv)-1)
    pbar = ProgressBar(widgets=widgets, maxval=len(studentexport_csv)-1)
    pbar.start()
    pbar_index = 0
    
    for student in studentexport_csv[1:]:
        # update the progress bar
        pbar.update(pbar_index)
        pbar_index += 1
        # start off assuming the folder needs to be created
        create_folder = True
        
        classOf = student[headerMap['headers']['ClassOf']]
        student_number = student[headerMap['headers']['Student_Number']]
        student_LastFirst = student[headerMap['headers']['LastFirst']]
        folder_name =  student_LastFirst + ' - ' + student_number
        studentFolders[student_number] = {}

        if classOfFolders[classOf]:
            classOf_id = classOfFolders[classOf]['id']
        else:
            logger.error('missing data for parent folder: Class Of-{}; skipping student'.format(classOf))
            studentFolders[student_number] = None
            continue
        
        
        # search for an existing folder
        logger.debug('searching for folder {} with in Class Of-{}'.format(folder_name, classOf))
        try:
            result = myDrive.search(name=folder_name, parents=classOfFolders[classOf]['id'], 
                                    mimeType='folder',
                                    teamdrive=teamdriveID,
                                    fields='webViewLink, id, mimeType, name, parents')
        except GDriveError as e:
            logger.error('failed to search for folder {} with error {}'.format(folder_name, e))
            studentFolders[student_number] = None
            # stop processing here
            continue
        except Exception as e:
            logger.error('fatal error: {}'.format(e))
            doExit(0)
            
        # check that a valid response was recieved, or skip over entry 
        try:
            if result.has_key('files'):
                pass
        except AttributeError as e:
            logger.warn('no valid information was returned for student: {}; see previous errors'.format(folder_name))
            studentFolders[student_number] = None
            # stop processing here
            continue
            
        # check that folder exists and is OK
        if len(result['files']) > 0:
            logger.debug('folder exists, taking no action')
            studentFolders[student_number] = result['files'][0]
            create_folder = False
        if len(result['files']) > 1:
            print 'There are multiple folders for student: {}; please consolodate them'.format(folder_name)
            create_folder = False
            
        
        if create_folder:
            logger.debug('creating folder: {}'.format(folder_name))
            try:
                result = myDrive.add(name=folder_name, parents=classOfFolders[classOf]['id'], mimeType='folder',
                                    fields='name, webViewLink, id, mimeType')
            except GDriveError as e:
                logger.error('error creating folder {}: {}'.format(folder_name, e))
                studentFolders[student_number] = None
            except Exception as e:
                logger.error('fatal error: {}'.format(e))
                doExit(0)
            
            # check the result is OK and record it
            if any(result):
                studentFolders[student_number] = result
            else:
                studentFolders[student_number] = None
                # stop processing here
                continue
            
        # append the LastFirst field
        studentFolders[student_number]['LastFirst'] = student_LastFirst

    # finish updating the pbar for Class Of- Folders
    pbar.finish()
    print 'Completed\n'
        
    return(studentFolders)

In [14]:
def doExit(exit_level=0, testing=False):
    logger.info('exiting before completion with exit code {}'.format(exit_level))
    if not testing:
        exit(0)    

In [15]:
#### version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))

studentexport_list = ['~/Downloads', '~/Desktop']
studentexport = None
studentexport_path = None


downloadlink = 'https://foo.foof.foo'


updateConfig = False


logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

teamdriveName = '**UNKNOWN**'

# for use this for doExit to prevent killing the Jupyter kernel when testing
testing = True

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = './gradefolders.txt'


    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logger.debug('current configuration:')
    logger.debug('\n{}'.format(config_dict))

# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account with access to the portfolio folder team drive and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
if not prompts.prompt_for_confirmation(question='Would you like to proceed?', default='y'):
    print 'Exiting'
    doExit(testing=testing)
    
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if not teamdriveID:
    logger.warn('Team Drive ID missing')
    try:
        teamdrive = getTeamDrive(myDrive)
    except GDriveError as e:
        logging.error('Error accessing team drives: e')
        doExit(testing=testing)
        
    if not teamdrive:
        logger.error('no team drives found')
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print('No Team Drives located; cannot proceed.')
        print('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        logger.error('exiting')
        doExit(testing=testing)
        
    try:
        teamdriveID = teamdrive['id']
        teamdriveName = teamdrive['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid team drive information found: {}'.format(e))
        logger.error('userinfo: {}'.format(myDrive.userinfo['emailAddress']))
        print 'No Team Drive was found. Do you have write access to a Team Drive with this account?'
        print 'You are authenticated as: {}'.format(myDrive.userinfo['emailAddress'])
        logger.error('exiting')
        doExit(testing=testing)
        
    print 'Using Team Drive: {}'.format(teamdriveName)    
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdriveName', teamdriveName)
    updateConfig = True
    
if not folderID:
    logger.warn('Folder ID missing')
    try:
        folder = getPortfolioFolder(myDrive)
    except GDriveError as e:
        logger.error('Could not search for folder: {}'.format(e))
        print ('There was a problem searching Team Drive for a folder')
        print ('You are authenticated as: {}'.format(myDrive.userinfo['emailAddress']))
        print ('If this is not correct do...')
    try:
        folderID = folder['id']
        folderName = folder['name']
    except (KeyError, TypeError) as e:
        logger.error('no valid folder information found {}'.format(e))
        logger.error('exiting')
        doExit(testing=testing)
    
    print 'Using Folder: {} on Team Drive: {}'.format(folderName, teamdriveName)
    myConfig.set('Main', 'foldername', folderName)
    myConfig.set('Main', 'folderid', folderID)
    updateConfig=True

if not os.path.exists(gradefolders):
    logger.error('failed to locate {}'.format(gradefolders))
    logger.error('exiting')
    print 'gradefolders.txt file is missing. Please reinstall the application.'
    print 'download link: {}'.format(downloadlink)
    doExit(testing=testing)

# get the gradefolder file
gradefolder_list = fileToList(gradefolders)

# check validity of gradefolder
if len(gradefolder_list) < 1:
    print 'grade folder file is corrupt. Please reinstall the application'
    doExit(testing=testing)


if updateConfig:
    configuration.create_config(cfgfile, myConfig)


# # get and validate path for student_export folder
attempt = 5
while not studentexport and (attempt > 0):
    logger.debug('remaining attempts to locate student export: {}'.format(attempt))
    attempt -= 1
    if not studentexport:
        # invoke menu for prompting user to choose a path to the studentexport file
        studentexport_path = getPathfromList(studentexport_list, 
                                             message='Please choose which folder contains the Student Export File.',
                                             default=studentexport_list[0])
    
    # move this pattern into the configuration file? 
    studentexport = chooseFile(path=studentexport_path, pattern='.*student.*export.*')
    logger.debug('student export file chosen: {}'.format(studentexport))
    if not os.access(studentexport, os.R_OK):
        print 'Could not read file: {}. Please choose another file'.format(studentexport)
        logger.error('file is unreadable: {}'.format(studentexport))
        studentexport = None

#TO DO - this looks odd. it appears that studentexport is pulled twice but unused.
# try commenting this entire block out. It does not look right.
if not studentexport:
    logger.error('no student export file; exiting')
    print 'Cannot proceed without a student export file. Exiting'
    doExit(testing=testing)
else:
    myConfig.set('Main', 'studentexport', studentexport)
    studentexport_list = fileToList(studentexport)
        
logger.debug('Student export file: {}'.format(studentexport))

# read the studentexport text file into a csv object
studentexport_csv = []
try: 
    with open(studentexport, 'rU') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            studentexport_csv.append(row)
except (OSError, IOError) as e:
    logging.critical('error reading file: {}\n{}'.format(studentexport,e))
    print 'Could not read file: {}; exiting.'.format(studentexport)
    doExit(testing=testing)

# parse and check validity of student export file
expected_headers = ['ClassOf', 'Student_Number', 'LastFirst']
headerMap = mapHeaders(file_csv=studentexport_csv, expected_headers=expected_headers)

if not headerMap or (len(headerMap['headers']) != len(expected_headers)):
    logging.error('cannot continue without valid header map; exiting')
    print('file: {} appears to not contain the expected header row (in any order): {}'.format(studentexport, expected_headers))
    if headerMap['missingheaders']:
        print('your file appears to be missing the field(s): {}'.format(headerMap['missingheaders']))
    print('please try to download a new student.export file before trying again; exiting')
    doExit(testing=testing)

logger.debug('asking to proceed with current configuration')
print '\nCurent Configuration:'
for section in myConfig.sections():
    print '[{}]'.format(section)
    for option in myConfig.options(section):
        print '{} = {}'.format(option, myConfig.get(section, option))
if not prompts.prompt_for_confirmation('Continue with the configuration above?', default='Y'):
    logger.debug('user chose to exit')
    doExit(testing=testing)
    
# start processing the student export file
student_folders = createFolders(myDrive=myDrive, parentFolder=folderID, teamdriveID=teamdriveID, 
                                studentexport_csv=studentexport_csv, header_map=headerMap )

# if not student_folders:
#     logger.error('Student folders were not created. Please try again. If this message persists, please check the logs')
#     doExit(testing=testing)



2018-09-23 21:10:46,307: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-09-23 21:10:46,309: [DEBUG: configuration.get_config] reading configuration file at: /Users/aaronciuffo/.config/portfolio_creator/portfolio_creator.ini
2018-09-23 21:10:46,310: [DEBUG: __main__.<module>] current configuration:
2018-09-23 21:10:46,311: [DEBUG: __main__.<module>] 
{'Main': {'folderid': '1-D7UNBes_skfkQ6oBettiBICiBcZmbvn', 'credentials': '~/.config/portfolio_creator/credentials/', 'foldername': 'Portfolios', 'teamdrivename': 'IT Blabla', 'teamdriveid': '0ACLfU8KeD_BHUk9PVA'}}
2018-09-23 21:10:46,312: [INFO: root.<module>] checking google credentials




You *may* be directed to a google web page that asks you to authorize this applicaiton.
Please choose an ASH account with access to the portfolio folder team drive and authorize this applicaiton.
When you are done, please close the newly created tab in your web browser and return to this window.
2018-09-23 21:10:46,314: [DEBUG: humanfr

2018-09-23 21:10:48,275: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:10:48,278: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/teamdrives?fields=teamDrives&alt=json
2018-09-23 21:10:48,665: [DEBUG: __main__.fileToList] inputfile = ./gradefolders.txt
2018-09-23 21:10:48,666: [DEBUG: __main__.fileToList] reading file: ./gradefolders.txt
2018-09-23 21:10:48,667: [DEBUG: __main__.<module>] remaining attempts to locate student export: 5
2018-09-23 21:10:48,668: [DEBUG: __main__.getPathfromList] path list: ['~/Downloads', '~/Desktop']
Please choose which folder contains the Student Export File.
2018-09-23 21:10:48,669: [DEBUG: humanfriendly.prompts.prompt_for_choice] Requesting interactive choice on terminal (options are '~/Desktop', '~/Downloads' and 'OTHER') ..

  1. ~/Desktop
  2. ~/Downloads (default choice)
  3. OTHER
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-23 21:10:51,655: [DEBUG:


  1. /Users/aaronciuffo/Downloads/student.export (1).text
  2. /Users/aaronciuffo/Downloads/student.export (2).text
  3. /Users/aaronciuffo/Downloads/student.export (3).text
  4. /Users/aaronciuffo/Downloads/student.export (4).text
  5. /Users/aaronciuffo/Downloads/student.export.text
  6. /Users/aaronciuffo/Downloads/student_export.text
  7. /Users/aaronciuffo/Downloads/Student_Export.txt
 
 Enter your choice as a number or unique substring (Control-C aborts): 4
2018-09-23 21:10:56,639: [DEBUG: humanfriendly.prompts.prompt_for_choice] Option ('/Users/aaronciuffo/Downloads/student.export (4).text') selected by numeric reply (4).
2018-09-23 21:10:56,640: [DEBUG: __main__.<module>] student export file chosen: /Users/aaronciuffo/Downloads/student.export (4).text
2018-09-23 21:10:56,640: [DEBUG: __main__.fileToList] inputfile = /Users/aaronciuffo/Downloads/student.export (4).text
2018-09-23 21:10:56,641: [DEBUG: __main__.fileToList] reading file: /Users/aaronciuffo/Downloads/student.expor


 Continue with the configuration above? [Y/n] 
2018-09-23 21:11:02,221: [DEBUG: humanfriendly.prompts.prompt_for_confirmation] Default choice selected by empty reply ('granted').
2018-09-23 21:11:02,222: [DEBUG: __main__.createFolders] creating folders at gdrive path: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:11:02,222: [DEBUG: __main__.createFolders] checking that portfolio folder exists and is writeable: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:11:02,223: [DEBUG: __main__.checkFolder] checking folder id: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:11:02,224: [DEBUG: __main__.checkFolder] checking properties for ID: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:11:02,225: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:11:02,226: [DEBUG: gdrive.gdrive.getprops] files().get(fileId=1-D7UNBes_skfkQ6oBettiBICiBcZmbvn, fields=capabilities,mimeType,name)
2018-09-23 21:11:02,236: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapi

2018-09-23 21:11:02,474: [DEBUG: __main__.checkFolder] this ID is a folder
2018-09-23 21:11:02,475: [DEBUG: __main__.checkFolder] this ID is a writable folder
2018-09-23 21:11:02,476: [DEBUG: __main__.createFolders] folder is OK
2018-09-23 21:11:02,476: [DEBUG: __main__.createFolders] gathering "Class Of" folders from student export file
2018-09-23 21:11:02,477: [DEBUG: __main__.createFolders] 3 "Class Of" folders found in student export file
2018-09-23 21:11:02,478: [DEBUG: __main__.createFolders] checking for existing "Class Of" folders in google drive


Checking for or creating 3 "Class Of-" folders in portfolio folder
2018-09-23 21:11:02,480: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:11:02,480: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2027" and trashed=False and mimeType="application/vnd.google-apps.folder", orderBy=createdTime, fields=files(name,createdTime,mimeType,parents,properties,id)))
2018-09-23 21:11:02,486: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28name%2CcreatedTime%2CmimeType%2Cparents%2Cproperties%2Cid%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2027%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json


In [37]:
# foo = createFolders(myDrive=myDrive, parentFolder='1Uzp9FFXNEc-_sO8ApVAaU08EY0ekmzwO', teamdriveID='0ADVhiLOX2_ufUk9PVA', studentexport_csv=studentexport_csv, header_map=headerMap )
foo = createFolders(myDrive=myDrive, parentFolder=folderID, teamdriveID=teamdriveID, studentexport_csv=studentexport_csv, header_map=headerMap )



2018-09-23 21:31:15,270: [DEBUG: __main__.createFolders] creating folders at gdrive path: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:31:15,272: [DEBUG: __main__.createFolders] checking that portfolio folder exists and is writeable: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:31:15,272: [DEBUG: __main__.checkFolder] checking folder id: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:31:15,273: [DEBUG: __main__.checkFolder] checking properties for ID: 1-D7UNBes_skfkQ6oBettiBICiBcZmbvn
2018-09-23 21:31:15,274: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:15,275: [DEBUG: gdrive.gdrive.getprops] files().get(fileId=1-D7UNBes_skfkQ6oBettiBICiBcZmbvn, fields=capabilities,mimeType,name)
2018-09-23 21:31:15,279: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files/1-D7UNBes_skfkQ6oBettiBICiBcZmbvn?fields=capabilities%2CmimeType%2Cname&alt=json&supportsTeamDrives=true
2018-09-23 21:31:15,607: [DEBUG: __main__.checkFolder]

Checking for or creating 3 "Class Of-" folders in portfolio folder
2018-09-23 21:31:15,613: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:15,614: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2027" and trashed=False and mimeType="application/vnd.google-apps.folder", orderBy=createdTime, fields=files(name,createdTime,mimeType,parents,properties,id)))
2018-09-23 21:31:15,619: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28name%2CcreatedTime%2CmimeType%2Cparents%2Cproperties%2Cid%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2027%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-23 21:31:15,929: [DEBUG: __main__.createFolders] folder found; checking if valid and wirteable
2018-09-23 21:31:15,930: [DEBUG: __main__.checkFolder] checki

2018-09-23 21:31:16,156: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:16,156: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2028" and trashed=False and mimeType="application/vnd.google-apps.folder", orderBy=createdTime, fields=files(name,createdTime,mimeType,parents,properties,id)))
2018-09-23 21:31:16,161: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28name%2CcreatedTime%2CmimeType%2Cparents%2Cproperties%2Cid%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2028%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-23 21:31:16,419: [DEBUG: __main__.createFolders] folder found; checking if valid and wirteable
2018-09-23 21:31:16,419: [DEBUG: __main__.checkFolder] checking folder id: 1uP2rGlS5QAeh6ApaifMDb34sxZ6sN8WM
2018-09-23 21:31:16

2018-09-23 21:31:16,736: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:16,737: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Class Of-2029" and trashed=False and mimeType="application/vnd.google-apps.folder", orderBy=createdTime, fields=files(name,createdTime,mimeType,parents,properties,id)))
2018-09-23 21:31:16,741: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28name%2CcreatedTime%2CmimeType%2Cparents%2Cproperties%2Cid%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Class+Of-2029%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-23 21:31:16,950: [DEBUG: __main__.createFolders] folder found; checking if valid and wirteable
2018-09-23 21:31:16,951: [DEBUG: __main__.checkFolder] checking folder id: 1WIGcKU0w9JDEYfADHSijduGuL2Yf06Nc
2018-09-23 21:31:16

Checking: 100% [====================================] Processed 3 Time: 0:00:01


...

2018-09-23 21:31:20,803: [DEBUG: __main__.createFolders] searching for folder Bogdaneris, Hugo - 504363 with in Class Of-2028
2018-09-23 21:31:20,804: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:20,805: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Bogdaneris, Hugo - 504363" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1uP2rGlS5QAeh6ApaifMDb34sxZ6sN8WM" in parents, orderBy=createdTime, fields=files(webViewLink,id,mimeType,name,parents)))
2018-09-23 21:31:20,811: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28webViewLink%2Cid%2CmimeType%2Cname%2Cparents%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Bogdaneris%2C+Hugo+-+504363%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221uP2rGlS5QAeh6ApaifMDb34sxZ6sN8WM%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=js

2018-09-23 21:31:21,289: [DEBUG: __main__.createFolders] searching for folder Borg, William - 503955 with in Class Of-2028
2018-09-23 21:31:21,290: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:21,291: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Borg, William - 503955" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1uP2rGlS5QAeh6ApaifMDb34sxZ6sN8WM" in parents, orderBy=createdTime, fields=files(webViewLink,id,mimeType,name,parents)))
2018-09-23 21:31:21,296: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28webViewLink%2Cid%2CmimeType%2Cname%2Cparents%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Borg%2C+William+-+503955%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221uP2rGlS5QAeh6ApaifMDb34sxZ6sN8WM%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-0

2018-09-23 21:31:21,663: [DEBUG: __main__.createFolders] searching for folder Sue, Sally - 000000 with in Class Of-2029
2018-09-23 21:31:21,664: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:21,665: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Sue, Sally - 000000" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1WIGcKU0w9JDEYfADHSijduGuL2Yf06Nc" in parents, orderBy=createdTime, fields=files(webViewLink,id,mimeType,name,parents)))
2018-09-23 21:31:21,670: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28webViewLink%2Cid%2CmimeType%2Cname%2Cparents%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Sue%2C+Sally+-+000000%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221WIGcKU0w9JDEYfADHSijduGuL2Yf06Nc%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json
2018-09-23 21:3

2018-09-23 21:31:22,029: [DEBUG: __main__.createFolders] searching for folder Washington, George - 001776 with in Class Of-2027
2018-09-23 21:31:22,029: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:31:22,030: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name = "Washington, George - 001776" and trashed=False and mimeType="application/vnd.google-apps.folder" and "1XoAHGRJGbfC3wgaS6XgT6-SEYM7N-_2Q" in parents, orderBy=createdTime, fields=files(webViewLink,id,mimeType,name,parents)))
2018-09-23 21:31:22,035: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&fields=files%28webViewLink%2Cid%2CmimeType%2Cname%2Cparents%29&corpora=teamDrive&supportsTeamDrives=true&q=name+%3D+%22Washington%2C+George+-+001776%22+and+trashed%3DFalse+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%221XoAHGRJGbfC3wgaS6XgT6-SEYM7N-_2Q%22+in+parents&teamDriveId=0ACLfU8KeD_BHUk9PVA&

Completed



In [23]:
myDrive.search(name='Misc Junk', fuzzy=True)

2018-09-23 21:16:08,486: [DEBUG: gdrive.gdrive.retry] attempt: 0
2018-09-23 21:16:08,488: [DEBUG: gdrive.gdrive.search] apicall: files().list(q=name contains "Misc Junk" and trashed=False, orderBy=createdTime, fields=files))
2018-09-23 21:16:08,493: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?q=name+contains+%22Misc+Junk%22+and+trashed%3DFalse&orderBy=createdTime&alt=json&fields=files


{u'files': [{u'capabilities': {u'canAddChildren': True,
    u'canChangeCopyRequiresWriterPermission': True,
    u'canChangeViewersCanCopyContent': True,
    u'canComment': True,
    u'canCopy': False,
    u'canDelete': True,
    u'canDownload': True,
    u'canEdit': True,
    u'canListChildren': True,
    u'canMoveItemIntoTeamDrive': True,
    u'canReadRevisions': False,
    u'canRemoveChildren': True,
    u'canRename': True,
    u'canShare': True,
    u'canTrash': True,
    u'canUntrash': True},
   u'copyRequiresWriterPermission': False,
   u'createdTime': u'2016-04-29T06:25:48.420Z',
   u'explicitlyTrashed': False,
   u'folderColorRgb': u'#8f8f8f',
   u'hasThumbnail': False,
   u'iconLink': u'https://drive-thirdparty.googleusercontent.com/16/type/application/vnd.google-apps.folder',
   u'id': u'0B9WTleJ1MzaYT2pieWNXYkZtZm8',
   u'isAppAuthorized': False,
   u'kind': u'drive#file',
   u'lastModifyingUser': {u'displayName': u'Aaron Ciuffo',
    u'emailAddress': u'aciuffo@ash.nl',
    u

In [17]:
# foo['100000'] = None
# failures = 0
# for each in foo:
#     if not foo[each]:
#         print 'student not processed due to errors: {}'.format(each)
#         failures += 1
# if failures > 0:
#     print 'Please re-run the program with the same data. If this message appears again, please check the logs [[path to logs]]'

In [18]:
# urlFormatter = '<a href={}>Student Portfolio for {}</a>'
# studentData_header = ['LastFirst', 'Student_Number', 'StudentURL']
# studentData_list = []
# studentData_list.append(studentData_header)

# for each in foo:
#     if foo[each]:
#         # LastFirst, Student Number, URL
#         studentData_list.append([foo[each]['LastFirst'], each, 
#                                  urlFormatter.format(foo[each]['webViewLink'], foo[each]['LastFirst'])])
        


In [19]:
# with open('./foo.tsv', mode='w') as tsv_file:
#     csv_writer = csv.writer(tsv_file, delimiter='\t')
#     csv_writer.writerows(studentData_list)